In [192]:
# load data
import glob
import os

# computing
import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt

# feature extraction
import cv2
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans

# modelling
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# evalutaion
from sklearn.metrics import classification_report

# load/save model
import pickle

# Load Data

In [22]:
# ukurang gambar 224x224
def import_image(file) :
    image = cv2.imread(file)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # image = cv2.resize(image, (128, 128), interpolation = cv2.INTER_AREA)
    return image

def label_image(character, length) :
    array = np.full(length, character)
    return array

def import_data(character, path) :
    file_path = glob.glob(path)
    images = np.array([import_image(file) for file in file_path])
    label = label_image(character, len(file_path))
    
    return images, label

In [24]:
hanacaraka = ('ba','ca','da','dha','ga','ha','ja','ka','la','ma','na','nga','nya','pa','ra','sa','ta','tha','wa','ya')

X_train = np.empty((0, 224, 224), np.uint8)
y_train = np.empty((0))
X_test = np.empty((0, 224, 224), np.uint8)
y_test = np.empty((0))

for character in hanacaraka:
    training_path = "../Dataset/train/" + character + "/*.png"
    testing_path = "../Dataset/val/" + character + "/*.png"
    
    temp_X_train, temp_y_train = import_data(character, training_path)
    temp_X_test, temp_y_test = import_data(character, testing_path)
    
    print("Ukuran karakter", character, "training:", temp_X_train.shape[0], "testing:", temp_X_test.shape[0])
    
    X_train = np.append(X_train, temp_X_train, axis=0)
    y_train = np.append(y_train, temp_y_train, axis=0)
    X_test = np.append(X_test, temp_X_test, axis=0)
    y_test = np.append(y_test, temp_y_test, axis=0)


Ukuran karakter ba training: 114 testing: 24
Ukuran karakter ca training: 108 testing: 24
Ukuran karakter da training: 108 testing: 24
Ukuran karakter dha training: 108 testing: 24
Ukuran karakter ga training: 108 testing: 24
Ukuran karakter ha training: 102 testing: 24
Ukuran karakter ja training: 108 testing: 24
Ukuran karakter ka training: 108 testing: 24
Ukuran karakter la training: 108 testing: 24
Ukuran karakter ma training: 108 testing: 24
Ukuran karakter na training: 108 testing: 24
Ukuran karakter nga training: 102 testing: 24
Ukuran karakter nya training: 108 testing: 24
Ukuran karakter pa training: 108 testing: 24
Ukuran karakter ra training: 108 testing: 24
Ukuran karakter sa training: 108 testing: 24
Ukuran karakter ta training: 108 testing: 24
Ukuran karakter tha training: 108 testing: 24
Ukuran karakter wa training: 108 testing: 24
Ukuran karakter ya training: 108 testing: 24


In [25]:
# Ukuran keseluruhan data
print("Training:", X_train.shape[0])
print("Testing:", X_test.shape[0])

Training: 2154
Testing: 480


# Feature Extraction

### Extract sift descriptors 

In [41]:
def extract_sift_features(list_image):

    image_descriptors = []
    sift = cv2.SIFT_create()
    for image in list_image:
        _, descriptor = sift.detectAndCompute(image, None)
        image_descriptors.append(descriptor)

    return image_descriptors

def split_all_descriptors(image_descriptors) :
    all_descriptors = []
    for descriptor in image_desctiptors:
        if descriptor is not None:
            for des in descriptor:
                all_descriptors.append(des)
    
    return np.array(all_descriptors)

In [42]:
image_descriptors = extract_sift_features(X_train)
all_descriptors = split_all_descriptors(image_desctiptors)

In [43]:
all_descriptors.shape

(108464, 128)

### Create bow of descriptor using cluster

In [112]:
def kmean_bow(all_descriptors, num_cluster):
    kmeans = KMeans(n_clusters = num_cluster)
    kmeans.fit(all_descriptors)

    bow_dict = kmeans.cluster_centers_

    #if not os.path.isfile('bow_dictionary.pkl'):
    pickle.dump(bow_dict, open('bow_dictionary.pkl', 'wb'))

    return bow_dict

In [113]:
num_cluster = 200
# bow = kmean_bow(all_descriptors, num_cluster)

### Create features from bow

In [158]:
def create_feature_bow(image_descriptors, bow, num_cluster):
    X_features = []

    for i in range(len(image_descriptors)):
        features = np.array([0] * num_cluster, dtype=float)

        if image_descriptors[i] is not None:
            distance = cdist(image_descriptors[i], bow)

            argmin = np.argmin(distance, axis = 1)

            for j in argmin:
                features[j] += 1.0
        X_features.append(features)

    return np.array(X_features)

In [159]:
X_train_feature = create_feature_bow(image_desctiptors, bow, num_cluster)
X_test_feature = create_feature_bow(extract_sift_features(X_test), bow, num_cluster)

print("Dimensi data training:", X_train_feature.shape)
print("Dimensi data testing:", X_test_feature.shape)

Dimensi data training: (2154, 200)
Dimensi data testing: (480, 200)


# Classification

In [173]:
# normalization
scaler = StandardScaler().fit(X_train_feature)
X_train_scale = scaler.transform(X_train_feature)
X_test_scale = scaler.transform(X_test_feature)

## KNN

In [174]:
knn_params = {'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]}
knn_cv = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=knn_params, cv=5, verbose=3)

In [175]:
knn_cv.fit(X_train_scale, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END .....................n_neighbors=1;, score=0.452 total time=   0.0s
[CV 2/5] END .....................n_neighbors=1;, score=0.399 total time=   0.0s
[CV 3/5] END .....................n_neighbors=1;, score=0.420 total time=   0.0s
[CV 4/5] END .....................n_neighbors=1;, score=0.497 total time=   0.0s
[CV 5/5] END .....................n_neighbors=1;, score=0.530 total time=   0.0s
[CV 1/5] END .....................n_neighbors=3;, score=0.436 total time=   0.0s
[CV 2/5] END .....................n_neighbors=3;, score=0.367 total time=   0.0s
[CV 3/5] END .....................n_neighbors=3;, score=0.390 total time=   0.0s
[CV 4/5] END .....................n_neighbors=3;, score=0.494 total time=   0.0s
[CV 5/5] END .....................n_neighbors=3;, score=0.502 total time=   0.0s
[CV 1/5] END .....................n_neighbors=5;, score=0.408 total time=   0.0s
[CV 2/5] END .....................n_neighbors=5;

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]},
             verbose=3)

In [176]:
print("Model terbaik:", knn_cv.best_estimator_)
print("Score terbaik:", knn_cv.best_score_)

Model terbaik: KNeighborsClassifier(n_neighbors=1)
Score terbaik: 0.4596427993309232


In [193]:
cv_results_df = pd.DataFrame(knn_cv.cv_results_)
cv_results_df = cv_results_df.sort_values(by=['rank_test_score'])
cv_results_df = (
    cv_results_df
    .set_index(cv_results_df["params"].apply(
        lambda x: "_".join(str(val) for val in x.values()))
    )
    .rename_axis('kernel')
)
cv_results_df[
    ['params', 'rank_test_score', 'mean_test_score', 'std_test_score']
]

,params,rank_test_score,mean_test_score,std_test_score
kernel,,,,
1,{'n_neighbors': 1},1,0.459643,0.048247
3,{'n_neighbors': 3},2,0.437820,0.054264
5,{'n_neighbors': 5},3,0.403934,0.054173
7,{'n_neighbors': 7},4,0.374233,0.059538
9,{'n_neighbors': 9},5,0.352875,0.054610
11,{'n_neighbors': 11},6,0.332439,0.045970
13,{'n_neighbors': 13},7,0.322224,0.046039
15,{'n_neighbors': 15},8,0.312003,0.042360
17,{'n_neighbors': 17},9,0.297610,0.038154


In [177]:
y_pred = knn_cv.predict(X_test_scale)

In [178]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          ba       0.55      0.25      0.34        24
          ca       0.80      0.33      0.47        24
          da       0.62      0.62      0.62        24
         dha       0.58      0.29      0.39        24
          ga       0.21      0.17      0.19        24
          ha       0.40      0.08      0.14        24
          ja       0.80      0.83      0.82        24
          ka       0.87      0.54      0.67        24
          la       0.29      0.21      0.24        24
          ma       0.75      0.62      0.68        24
          na       0.78      0.29      0.42        24
         nga       0.55      0.67      0.60        24
         nya       1.00      0.04      0.08        24
          pa       0.17      0.96      0.29        24
          ra       0.14      0.38      0.20        24
          sa       0.62      0.67      0.64        24
          ta       0.62      0.21      0.31        24
         tha       1.00    

### SVM

In [203]:
svm_params = {
    'C': [0.5, 1, 3, 6, 10],
    'gamma': ['scale', 'auto', 0.001, 0.003, 0.006]
}
svm_cv = GridSearchCV(estimator=SVC(kernel='rbf'), param_grid=svm_params, cv=5, verbose=3)

In [204]:
svm_cv.fit(X_train_scale, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ................C=0.5, gamma=scale;, score=0.738 total time=   0.4s
[CV 2/5] END ................C=0.5, gamma=scale;, score=0.687 total time=   0.3s
[CV 3/5] END ................C=0.5, gamma=scale;, score=0.701 total time=   0.3s
[CV 4/5] END ................C=0.5, gamma=scale;, score=0.861 total time=   0.4s
[CV 5/5] END ................C=0.5, gamma=scale;, score=0.840 total time=   0.4s
[CV 1/5] END .................C=0.5, gamma=auto;, score=0.735 total time=   0.4s
[CV 2/5] END .................C=0.5, gamma=auto;, score=0.687 total time=   0.3s
[CV 3/5] END .................C=0.5, gamma=auto;, score=0.701 total time=   0.3s
[CV 4/5] END .................C=0.5, gamma=auto;, score=0.861 total time=   0.4s
[CV 5/5] END .................C=0.5, gamma=auto;, score=0.840 total time=   0.4s
[CV 1/5] END ................C=0.5, gamma=0.001;, score=0.726 total time=   0.3s
[CV 2/5] END ................C=0.5, gamma=0.001

[CV 2/5] END .................C=10, gamma=scale;, score=0.703 total time=   0.4s
[CV 3/5] END .................C=10, gamma=scale;, score=0.733 total time=   0.4s
[CV 4/5] END .................C=10, gamma=scale;, score=0.870 total time=   0.4s
[CV 5/5] END .................C=10, gamma=scale;, score=0.867 total time=   0.4s
[CV 1/5] END ..................C=10, gamma=auto;, score=0.768 total time=   0.4s
[CV 2/5] END ..................C=10, gamma=auto;, score=0.705 total time=   0.4s
[CV 3/5] END ..................C=10, gamma=auto;, score=0.735 total time=   0.4s
[CV 4/5] END ..................C=10, gamma=auto;, score=0.870 total time=   0.4s
[CV 5/5] END ..................C=10, gamma=auto;, score=0.867 total time=   0.4s
[CV 1/5] END .................C=10, gamma=0.001;, score=0.766 total time=   0.2s
[CV 2/5] END .................C=10, gamma=0.001;, score=0.701 total time=   0.2s
[CV 3/5] END .................C=10, gamma=0.001;, score=0.752 total time=   0.2s
[CV 4/5] END ...............

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.5, 1, 3, 6, 10],
                         'gamma': ['scale', 'auto', 0.001, 0.003, 0.006]},
             verbose=3)

In [205]:
print("Model terbaik:", svm_cv.best_estimator_)
print("Score terbaik:", svm_cv.best_score_)

Model terbaik: SVC(C=3, gamma=0.003)
Score terbaik: 0.799477688447634


In [206]:
cv_results_df = pd.DataFrame(svm_cv.cv_results_)
cv_results_df = cv_results_df.sort_values(by=['rank_test_score'])
cv_results_df = (
    cv_results_df
    .set_index(cv_results_df["params"].apply(
        lambda x: "_".join(str(val) for val in x.values()))
    )
    .rename_axis('kernel')
)
cv_results_df[
    ['params', 'rank_test_score', 'mean_test_score', 'std_test_score']
]

,params,rank_test_score,mean_test_score,std_test_score
kernel,,,,
3_0.003,"{'C': 3, 'gamma': 0.003}",1,0.799478,0.060956
6_0.003,"{'C': 6, 'gamma': 0.003}",2,0.799016,0.062437
10_0.003,"{'C': 10, 'gamma': 0.003}",3,0.794833,0.058596
3_auto,"{'C': 3, 'gamma': 'auto'}",4,0.793912,0.074332
3_scale,"{'C': 3, 'gamma': 'scale'}",5,0.792055,0.074637
6_auto,"{'C': 6, 'gamma': 'auto'}",6,0.790658,0.069559
10_auto,"{'C': 10, 'gamma': 'auto'}",7,0.789266,0.067866
6_scale,"{'C': 6, 'gamma': 'scale'}",7,0.789266,0.070648
10_scale,"{'C': 10, 'gamma': 'scale'}",9,0.787874,0.068954


In [207]:
y_pred = svm_cv.predict(X_test_scale)

In [208]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          ba       0.65      0.71      0.68        24
          ca       0.88      0.62      0.73        24
          da       0.75      1.00      0.86        24
         dha       0.90      0.75      0.82        24
          ga       1.00      0.54      0.70        24
          ha       0.47      0.29      0.36        24
          ja       0.85      0.96      0.90        24
          ka       0.79      0.96      0.87        24
          la       0.39      0.50      0.44        24
          ma       0.74      0.96      0.84        24
          na       1.00      0.46      0.63        24
         nga       0.78      0.88      0.82        24
         nya       0.86      0.79      0.83        24
          pa       0.64      0.96      0.77        24
          ra       0.78      0.75      0.77        24
          sa       0.72      0.96      0.82        24
          ta       0.95      0.79      0.86        24
         tha       1.00    